In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

from ipynb.fs.full.UtilCollections import TEAM_TO_ABBR
from ipynb.fs.full.UtilFunctions import format_season

In [10]:
def get_team_stats(seasons, playoffs=False):
    selector = "per_game-team"
    regular_or_playoffs="leagues"
    team_column = "Team"
    
    if playoffs:
        regular_or_playoffs = "playoffs"
        
    for season in seasons:
        table_position = 2
        if season >= 2016:
            table_position = 4
        if playoffs:
            table_position = -5
            
        #Reads the html data and turns it into a DataFrame
        url = f'https://www.basketball-reference.com/{regular_or_playoffs}/NBA_{season}.html#{selector}'
        html = urlopen(url)
        soup = BeautifulSoup(html)
        tables = soup.find_all('table')
        df = pd.read_html(str(tables[table_position]))[0]
        #Eliminates league abverage row
        league_avg_index = df[df[team_column] == 'League Average'].index[0]
        df = df[:league_avg_index]

        #Marks wether teams made the playoffs or not
        df["Made_Playoffs"] = False
        
        for team in df[team_column]:
            if '*' in team:
                df.loc[df[team_column] == team, ["Made_Playoffs"]] = True
                
        df[team_column] = df[team_column].apply(lambda x: x.replace('*', '').upper())
        df[team_column] = df[team_column].apply(lambda x: TEAM_TO_ABBR[x])
        df['Rk'] = df['Rk'].apply(lambda x: int(x))
        df = df.set_index('Rk')
        
        first_year, second_year = format_season(season)

        csv_file_name = "DataCollection/Team_Stats/team_stats_{0}-{1}.csv".format(first_year, second_year)
        if playoffs:
            csv_file_name = "DataCollection/Team_Stats_Playoffs/team_stats_playoffs{0}-{1}.csv".format(first_year, second_year)

        df.to_csv(csv_file_name, index=False)

In [13]:
def get_opponent_stats(seasons, playoffs=False):
    selector = "per_game-opponent"
    regular_or_playoffs="leagues"
    team_column = "Team"
    
    if playoffs:
        regular_or_playoffs = "playoffs"
        
    for season in seasons:
        table_position = 3
        if season >= 2016:
            table_position = 5
        if playoffs:
            table_position = -4
        url = f'https://www.basketball-reference.com/{regular_or_playoffs}/NBA_{season}.html#{selector}'
        soup = BeautifulSoup(urlopen(url))
        tables = soup.find_all('table')
        df = pd.read_html(str(tables[table_position]))[0]
        league_avg_index = df[df[team_column] == 'League Average'].index[0]
        df = df[:league_avg_index]
        
        df["Made_Playoffs"] = False
        for team in df[team_column]:
            if '*' in team:
                df.loc[df[team_column] == team, ["Made_Playoffs"]] = True
                
        df[team_column] = df[team_column].apply(lambda x: x.replace('*', '').upper())
        df[team_column] = df[team_column].apply(lambda x: TEAM_TO_ABBR[x])
        df['Rk'] = df['Rk'].apply(lambda x: int(x))
        df = df.set_index('Rk')
        
        static_columns = [team_column, "G", "MP"]
        opp_columns = static_columns
        for column in df.columns:
            if column not in static_columns:
                column = "OPP_"+column
                opp_columns.append(column)
                
        df.columns = opp_columns
    

        first_year, second_year = format_season(season)

        csv_file_name = "DataCollection/Opponent_Stats/opponent_stats_{0}-{1}.csv".format(first_year, second_year)
        if playoffs:
            csv_file_name = "DataCollection/Opponent_Stats_Playoffs/opponent_stats_playoffS_{0}-{1}.csv".format(first_year, second_year)

        df.to_csv(csv_file_name, index=False)

In [26]:
def get_advanced_team_stats(seasons, playoffs=False):
    selector = "div_advanced-team"
    regular_or_playoffs="leagues"
    team_column = "Team"
    
    if playoffs:
        regular_or_playoffs = "playoffs"
        team_column = "Tm"
            
    for season in seasons:
        table_position = 8
        if season >= 2016:
            table_position = 10
        if playoffs:
            table_position = -1
        url = f'https://www.basketball-reference.com/{regular_or_playoffs}/NBA_{season}.html#{selector}'
        soup = BeautifulSoup(urlopen(url))
        tables = soup.find_all('table')
        df = pd.read_html(str(tables[table_position]))[0]

        df.columns = list(map(lambda x: x[1], list(df.columns)))
        league_avg_index = df[df[team_column] == 'League Average'].index[0]
        df = df[:league_avg_index]

        df["Made_Playoffs"] = False
        for team in df[team_column]:
            if '*' in team:
                df.loc[df[team_column] == team, ["Made_Playoffs"]] = True

        df[team_column] = df[team_column].apply(lambda x: x.replace('*', '').upper())
        df[team_column] = df[team_column].apply(lambda x: TEAM_TO_ABBR[x])
        df['Rk'] = df['Rk'].apply(lambda x: int(x))
        df = df.set_index('Rk')
        df = df.loc[:, ~df.columns.str.contains("Unnamed")]

        columns = list(df.columns)
        if not playoffs:
            for i in range(16, 24):
                column = columns[i]
                if i < 20:
                    columns[i] = "Off_"+column
                else:
                    columns[i] = "Def_"+column
        else:
            for i in range(14, 22):
                column = columns[i]
                if i < 18:
                    columns[i] = "Off_"+column
                else:
                    columns[i] = "Def_"+column
        df.columns = columns

        first_year, second_year = format_season(season)

        csv_file_name = "DataCollection/Advanced_Team_Stats/adv_team_stats_{0}-{1}.csv".format(first_year, second_year)
        if playoffs:
             csv_file_name = "DataCollection/Advanced_Team_Stats_Playoffs/adv_team_stats_playoffs_{0}-{1}.csv".format(first_year, second_year)
        
        df.to_csv(csv_file_name, index=False)

In [6]:
seasons = range(2000, 2023)

get_team_stats(seasons)

In [11]:
get_team_stats(seasons, True)

2000
      Rk                    Team   G     MP    FG   FGA    FG%   3P   3PA  \
0    1.0      Los Angeles Lakers  23  241.1  37.3  80.4  0.465  5.4  15.4   
1    2.0          Indiana Pacers  23  241.1  35.0  78.9  0.443  7.1  18.8   
2    3.0        Sacramento Kings   5  240.0  33.6  81.0  0.415  6.6  17.4   
3    4.0         Milwaukee Bucks   5  240.0  37.8  82.6  0.458  3.6  11.6   
4    5.0      Philadelphia 76ers  10  242.5  34.2  80.0  0.428  3.9  12.3   
5    6.0     Seattle Supersonics   5  240.0  34.8  79.6  0.437  7.0  16.4   
6    7.0  Portland Trail Blazers  16  240.0  32.4  73.1  0.444  5.4  14.6   
7    8.0       Charlotte Hornets   4  246.3  32.3  75.0  0.430  4.5  15.0   
8    9.0               Utah Jazz  10  240.0  33.1  74.6  0.444  4.6  12.3   
9   10.0            Phoenix Suns   9  240.0  33.4  82.2  0.407  4.7  16.8   
10  11.0  Minnesota Timberwolves   4  240.0  34.3  76.5  0.448  3.0   9.8   
11  12.0         New York Knicks  16  241.6  31.2  72.4  0.431  4.5  11

      Rk                    Team   G     MP    FG   FGA    FG%   3P   3PA  \
0    1.0        Sacramento Kings  12  244.2  39.3  85.5  0.459  6.8  18.4   
1    2.0        Dallas Mavericks  20  242.5  38.7  86.1  0.450  9.2  23.2   
2    3.0  Portland Trail Blazers   7  240.0  36.4  79.0  0.461  6.3  17.0   
3    4.0  Minnesota Timberwolves   6  244.2  37.5  81.0  0.463  4.7  14.0   
4    5.0      Los Angeles Lakers  12  242.1  37.3  83.3  0.448  7.1  20.0   
5    6.0         Milwaukee Bucks   6  244.2  36.5  79.5  0.459  6.3  17.0   
6    7.0       San Antonio Spurs  24  241.0  33.8  76.8  0.441  5.3  15.2   
7    8.0          Boston Celtics  10  247.5  33.4  79.1  0.422  8.6  24.3   
8    9.0         New Jersey Nets  20  243.8  34.2  80.6  0.425  4.7  13.3   
9   10.0               Utah Jazz   5  240.0  33.8  77.8  0.434  1.8   6.8   
10  11.0     New Orleans Hornets   6  240.0  33.8  77.2  0.438  6.3  17.2   
11  12.0      Philadelphia 76ers  12  244.2  33.7  75.9  0.443  3.8  10.7   

      Rk                  Team   G     MP    FG   FGA    FG%   3P   3PA  \
0    1.0          Phoenix Suns  20  245.0  40.1  83.1  0.482  9.2  23.5   
1    2.0  Los Angeles Clippers  12  244.2  39.7  80.8  0.491  4.5  11.9   
2    3.0     San Antonio Spurs  13  245.8  36.5  74.1  0.492  6.0  15.3   
3    4.0    Washington Wizards   6  248.3  35.3  78.7  0.449  6.2  19.2   
4    5.0    Los Angeles Lakers   7  247.1  37.7  83.1  0.454  7.1  21.4   
5    6.0      Dallas Mavericks  23  244.3  36.1  80.1  0.451  4.8  15.1   
6    7.0         Chicago Bulls   6  240.0  34.0  76.3  0.445  7.8  19.8   
7    8.0       Milwaukee Bucks   5  240.0  37.8  80.2  0.471  5.2  12.4   
8    9.0      Sacramento Kings   6  244.2  34.7  78.2  0.443  5.2  15.7   
9   10.0            Miami Heat  23  241.1  34.8  72.4  0.481  6.3  19.1   
10  11.0       New Jersey Nets  11  240.0  34.3  75.8  0.452  5.4  17.5   
11  12.0       Detroit Pistons  18  240.0  32.1  74.3  0.432  6.9  18.5   
12  13.0   Cleveland Cava

      Rk                    Team   G     MP    FG   FGA    FG%   3P   3PA  \
0    1.0           Chicago Bulls   7  265.0  38.4  87.1  0.441  7.3  19.3   
1    2.0          Denver Nuggets  16  240.0  37.3  78.4  0.475  8.1  21.1   
2    3.0      Los Angeles Lakers  23  242.2  37.4  80.5  0.465  7.0  18.4   
3    4.0          Boston Celtics  14  252.5  37.4  84.1  0.445  6.7  18.6   
4    5.0        Dallas Mavericks  10  240.0  35.6  75.7  0.470  7.3  20.6   
5    6.0     Cleveland Cavaliers  14  241.8  34.8  75.1  0.463  6.6  19.8   
6    7.0               Utah Jazz   5  240.0  35.0  81.0  0.432  5.0  14.6   
7    8.0           Orlando Magic  24  243.1  34.5  75.4  0.457  8.4  23.3   
8    9.0         Houston Rockets  13  240.0  34.0  76.1  0.447  6.6  20.2   
9   10.0       San Antonio Spurs   5  240.0  35.0  76.6  0.457  5.8  17.6   
10  11.0      Philadelphia 76ers   6  240.0  34.0  75.0  0.453  5.8  15.8   
11  12.0              Miami Heat   7  240.0  31.3  69.6  0.450  7.1  20.0   

      Rk                   Team   G     MP    FG   FGA    FG%   3P   3PA  \
0    1.0      San Antonio Spurs  14  240.0  37.9  78.9  0.481  9.4  22.6   
1    2.0  Oklahoma City Thunder  20  240.0  37.2  80.0  0.465  6.8  19.2   
2    3.0             Miami Heat  23  242.2  35.0  76.0  0.460  6.8  19.7   
3    4.0         Denver Nuggets   7  240.0  38.4  89.3  0.430  5.3  19.0   
4    5.0     Los Angeles Lakers  12  240.0  35.2  83.2  0.423  5.3  17.3   
5    6.0       Dallas Mavericks   4  240.0  31.5  78.0  0.404  8.0  21.5   
6    7.0      Memphis Grizzlies   7  243.6  32.6  76.3  0.427  3.4  11.9   
7    8.0         Indiana Pacers  11  242.3  33.7  79.5  0.424  5.7  17.9   
8    9.0   Los Angeles Clippers  11  242.3  34.5  75.3  0.458  6.2  16.4   
9   10.0         Boston Celtics  20  243.8  33.8  77.2  0.438  4.7  16.3   
10  11.0              Utah Jazz   4  240.0  32.8  85.8  0.382  2.3  11.3   
11  12.0     Philadelphia 76ers  13  240.0  32.1  78.1  0.411  4.0  13.2   
12  13.0    

      Rk                    Team   G     MP    FG   FGA    FG%    3P   3PA  \
0    1.0        Dallas Mavericks   5  240.0  41.6  92.0  0.452   7.6  24.4   
1    2.0         Houston Rockets  17  240.0  37.4  83.4  0.448   9.5  27.5   
2    3.0    Los Angeles Clippers  14  241.8  39.5  84.8  0.466   8.4  25.4   
3    4.0       San Antonio Spurs   7  243.6  38.3  85.3  0.449   9.6  26.1   
4    5.0   Golden State Warriors  21  243.6  38.2  84.2  0.453  11.4  30.5   
5    6.0      Washington Wizards  10  242.5  37.3  84.5  0.441   9.4  23.3   
6    7.0    New Orleans Pelicans   4  246.3  38.0  84.3  0.451   6.8  21.3   
7    8.0     Cleveland Cavaliers  20  243.8  34.7  82.7  0.420   9.8  28.7   
8    9.0           Atlanta Hawks  16  243.1  36.9  85.6  0.431   9.1  28.6   
9   10.0           Brooklyn Nets   6  244.2  36.2  83.2  0.435   8.5  24.0   
10  11.0         Toronto Raptors   4  246.3  36.8  88.3  0.416   7.5  25.0   
11  12.0           Chicago Bulls  12  244.2  35.1  83.8  0.419  

      Rk                    Team   G     MP    FG   FGA    FG%    3P   3PA  \
0    1.0   Golden State Warriors  21  241.2  41.2  86.0  0.480  11.2  31.4   
1    2.0    New Orleans Pelicans   9  240.0  42.9  89.8  0.478   9.9  27.0   
2    3.0      Philadelphia 76ers  10  242.5  38.7  88.5  0.437   9.9  29.3   
3    4.0         Toronto Raptors  10  242.5  38.9  82.3  0.473  10.4  26.7   
4    5.0      Washington Wizards   6  240.0  39.7  85.2  0.466   8.0  21.8   
5    6.0  Portland Trail Blazers   4  240.0  41.3  91.0  0.453  10.8  32.3   
6    7.0         Houston Rockets  17  240.0  37.2  85.3  0.436  13.4  39.8   
7    8.0              Miami Heat   5  240.0  36.6  82.8  0.442  10.8  27.6   
8    9.0               Utah Jazz  11  240.0  37.7  82.9  0.455  10.7  29.9   
9   10.0         Milwaukee Bucks   7  243.6  39.1  78.3  0.500   9.6  25.7   
10  11.0  Minnesota Timberwolves   5  240.0  38.0  82.6  0.460   9.0  21.8   
11  12.0          Boston Celtics  19  242.6  36.4  83.3  0.437  

      Rk                    Team   G     MP    FG   FGA    FG%    3P   3PA  \
0    1.0  Portland Trail Blazers   6  248.3  42.0  88.3  0.475  16.2  39.2   
1    2.0               Utah Jazz  11  240.0  39.7  84.2  0.472  18.0  43.6   
2    3.0      Philadelphia 76ers  12  240.0  42.5  85.8  0.496  11.2  28.4   
3    4.0       Memphis Grizzlies   5  240.0  44.2  94.4  0.468  10.4  31.6   
4    5.0          Denver Nuggets  10  245.0  42.1  92.2  0.457  14.7  38.5   
5    6.0           Brooklyn Nets  12  242.1  40.7  86.2  0.472  13.6  35.6   
6    7.0          Boston Celtics   5  240.0  38.4  88.2  0.435  12.8  36.6   
7    8.0    Los Angeles Clippers  19  240.0  39.1  81.9  0.477  13.8  36.4   
8    9.0         Milwaukee Bucks  23  242.2  42.1  91.1  0.462  11.2  35.0   
9   10.0      Washington Wizards   5  240.0  40.6  88.6  0.458   7.0  24.2   
10  11.0            Phoenix Suns  22  240.0  40.9  85.0  0.481  11.2  29.7   
11  12.0           Atlanta Hawks  18  240.0  38.9  86.8  0.449  

In [12]:
get_opponent_stats(seasons)

In [14]:
get_opponent_stats(seasons, True)

      Rk                    Team   G     MP    FG   FGA    FG%   3P   3PA  \
0    1.0              Miami Heat  10  242.5  28.8  71.3  0.404  3.9  11.4   
1    2.0       San Antonio Spurs   4  240.0  33.3  83.8  0.397  4.0  14.8   
2    3.0         New York Knicks  16  241.6  30.4  75.4  0.404  5.8  17.4   
3    4.0  Minnesota Timberwolves   4  240.0  31.3  71.8  0.436  5.8  15.5   
4    5.0         Toronto Raptors   3  240.0  30.3  67.0  0.453  5.3  11.3   
5    6.0  Portland Trail Blazers  16  240.0  32.4  72.9  0.445  4.8  14.4   
6    7.0         Detroit Pistons   3  240.0  34.3  71.7  0.479  4.7  11.3   
7    8.0            Phoenix Suns   9  240.0  31.9  77.3  0.412  3.7  12.3   
8    9.0               Utah Jazz  10  240.0  33.6  75.8  0.443  6.0  15.1   
9   10.0       Charlotte Hornets   4  246.3  34.3  80.0  0.428  6.0  15.0   
10  11.0         Milwaukee Bucks   5  240.0  34.0  81.6  0.417  4.8  16.2   
11  12.0     Seattle Supersonics   5  240.0  36.0  76.8  0.469  6.0  13.0   

      Rk                    Team   G     MP    FG   FGA    FG%   3P   3PA  \
0    1.0         Detroit Pistons  17  242.9  32.9  76.0  0.433  3.9  12.0   
1    2.0       San Antonio Spurs  24  241.0  33.0  81.6  0.404  5.8  17.5   
2    3.0         New Jersey Nets  20  243.8  32.7  76.5  0.427  5.6  16.1   
3    4.0            Phoenix Suns   6  244.2  31.0  76.8  0.403  5.3  17.7   
4    5.0      Philadelphia 76ers  12  244.2  31.8  75.2  0.424  7.4  20.2   
5    6.0     New Orleans Hornets   6  240.0  33.5  75.7  0.443  4.3  12.3   
6    7.0           Orlando Magic   7  240.0  32.0  75.7  0.423  6.0  19.1   
7    8.0          Boston Celtics  10  247.5  32.4  78.8  0.411  5.9  16.7   
8    9.0          Indiana Pacers   6  244.2  33.8  78.5  0.431  9.5  24.8   
9   10.0      Los Angeles Lakers  12  242.1  37.3  79.9  0.467  5.3  13.2   
10  11.0  Portland Trail Blazers   7  240.0  36.9  80.7  0.457  8.1  20.0   
11  12.0         Milwaukee Bucks   6  244.2  36.5  79.0  0.462  5.0  14.3   

      Rk                  Team   G     MP    FG   FGA    FG%   3P   3PA  \
0    1.0       Detroit Pistons  18  240.0  33.8  72.9  0.464  4.4  13.5   
1    2.0            Miami Heat  23  241.1  33.0  76.8  0.429  6.2  19.2   
2    3.0        Indiana Pacers   6  240.0  33.7  75.2  0.448  4.7  16.2   
3    4.0   Cleveland Cavaliers  13  243.8  32.8  75.2  0.437  6.5  17.8   
4    5.0        Denver Nuggets   5  240.0  34.2  74.0  0.462  4.6  12.8   
5    6.0       New Jersey Nets  11  240.0  32.9  70.6  0.466  7.2  19.7   
6    7.0      Dallas Mavericks  23  244.3  34.8  74.0  0.470  5.3  15.4   
7    8.0     Memphis Grizzlies   4  246.3  34.8  76.8  0.453  5.3  14.8   
8    9.0     San Antonio Spurs  13  245.8  35.8  78.5  0.456  4.6  13.4   
9   10.0         Chicago Bulls   6  240.0  36.5  78.7  0.464  7.8  23.2   
10  11.0    Washington Wizards   6  248.3  37.3  81.5  0.458  5.0  14.3   
11  12.0  Los Angeles Clippers  12  244.2  37.8  86.8  0.435  7.7  22.6   
12  13.0          Phoenix

      Rk                    Team   G     MP    FG   FGA    FG%   3P   3PA  \
0    1.0              Miami Heat   7  240.0  31.0  73.4  0.422  6.1  18.0   
1    2.0     Cleveland Cavaliers  14  241.8  32.4  74.9  0.433  7.0  19.1   
2    3.0           Atlanta Hawks  11  240.0  32.6  70.5  0.463  7.3  19.9   
3    4.0  Portland Trail Blazers   6  240.0  34.8  73.7  0.473  6.5  17.5   
4    5.0         Houston Rockets  13  240.0  35.3  78.5  0.450  5.8  16.1   
5    6.0         Detroit Pistons   4  240.0  32.0  69.5  0.460  5.3  17.0   
6    7.0           Orlando Magic  24  243.1  35.4  78.7  0.450  6.0  18.0   
7    8.0      Los Angeles Lakers  23  242.2  33.7  78.6  0.429  6.6  20.8   
8    9.0      Philadelphia 76ers   6  240.0  35.3  76.0  0.465  7.5  21.7   
9   10.0       San Antonio Spurs   5  240.0  34.2  73.8  0.463  7.8  20.4   
10  11.0          Denver Nuggets  16  240.0  33.9  75.7  0.448  6.8  19.9   
11  12.0          Boston Celtics  14  252.5  36.4  81.7  0.446  7.6  21.2   

      Rk                   Team   G     MP    FG   FGA    FG%   3P   3PA  \
0    1.0          Chicago Bulls   6  240.0  31.7  77.8  0.407  3.3  13.5   
1    2.0          Atlanta Hawks   6  244.2  33.2  76.7  0.433  4.3  14.7   
2    3.0     Philadelphia 76ers  13  240.0  33.5  77.9  0.430  4.4  14.8   
3    4.0         Boston Celtics  20  243.8  33.4  78.2  0.427  5.8  17.9   
4    5.0         Indiana Pacers  11  242.3  33.3  77.5  0.430  6.2  19.4   
5    6.0             Miami Heat  23  242.2  33.5  77.7  0.431  5.5  18.4   
6    7.0      Memphis Grizzlies   7  243.6  33.3  73.7  0.452  6.1  17.6   
7    8.0          Orlando Magic   5  245.0  36.4  81.8  0.445  5.8  18.2   
8    9.0   Los Angeles Clippers  11  242.3  34.7  76.5  0.454  5.6  15.5   
9   10.0      San Antonio Spurs  14  240.0  36.4  81.7  0.445  5.7  15.4   
10  11.0         Denver Nuggets   7  240.0  37.0  86.9  0.426  6.1  19.4   
11  12.0        New York Knicks   5  240.0  33.0  72.4  0.456  8.0  21.8   
12  13.0  Ok

      Rk                    Team   G     MP    FG   FGA    FG%    3P   3PA  \
0    1.0           Chicago Bulls  12  244.2  34.2  84.0  0.407   7.6  22.6   
1    2.0     Cleveland Cavaliers  20  243.8  34.9  83.1  0.419   7.9  25.5   
2    3.0   Golden State Warriors  21  243.6  34.8  83.4  0.417   6.8  22.4   
3    4.0       Memphis Grizzlies  11  240.0  35.1  80.5  0.436  10.4  27.8   
4    5.0      Washington Wizards  10  242.5  36.8  87.1  0.423   8.2  25.8   
5    6.0           Atlanta Hawks  16  243.1  36.6  85.0  0.430   9.4  25.2   
6    7.0  Portland Trail Blazers   5  240.0  37.2  85.0  0.438   4.6  12.4   
7    8.0         Milwaukee Bucks   6  248.3  36.5  83.5  0.437  11.2  28.0   
8    9.0           Brooklyn Nets   6  244.2  38.3  87.0  0.441  11.2  31.5   
9   10.0       San Antonio Spurs   7  243.6  38.7  83.0  0.466   7.0  21.0   
10  11.0          Boston Celtics   4  240.0  35.0  80.0  0.438   9.8  30.5   
11  12.0    Los Angeles Clippers  14  241.8  37.6  84.4  0.446  

      Rk                    Team   G     MP    FG   FGA    FG%    3P   3PA  \
0    1.0          Indiana Pacers   7  240.0  33.1  76.3  0.434  10.4  32.4   
1    2.0   Golden State Warriors  21  241.2  37.0  87.4  0.423  10.2  32.5   
2    3.0          Boston Celtics  19  242.6  37.5  81.4  0.461   9.2  27.5   
3    4.0         Milwaukee Bucks   7  243.6  37.0  83.6  0.443  10.1  29.0   
4    5.0     Cleveland Cavaliers  22  242.3  38.3  82.0  0.467  10.2  29.5   
5    6.0         Houston Rockets  17  240.0  38.5  82.6  0.465  10.6  27.9   
6    7.0               Utah Jazz  11  240.0  37.9  86.6  0.438  11.7  32.3   
7    8.0      Philadelphia 76ers  10  242.5  37.1  83.2  0.446  11.4  29.9   
8    9.0   Oklahoma City Thunder   6  240.0  38.2  84.2  0.453  11.3  31.5   
9   10.0       San Antonio Spurs   5  240.0  39.6  82.8  0.478   9.4  28.0   
10  11.0      Washington Wizards   6  240.0  39.2  82.8  0.473  11.0  26.8   
11  12.0  Minnesota Timberwolves   5  240.0  39.2  88.0  0.445  

      Rk                    Team   G     MP    FG   FGA    FG%    3P   3PA  \
0    1.0      Los Angeles Lakers   6  240.0  37.5  81.8  0.458  11.7  32.5   
1    2.0         New York Knicks   5  240.0  37.6  84.8  0.443  12.8  35.6   
2    3.0            Phoenix Suns  22  240.0  38.1  86.6  0.440  11.5  34.5   
3    4.0         Milwaukee Bucks  23  242.2  39.6  87.2  0.454  12.0  34.2   
4    5.0           Brooklyn Nets  12  242.1  39.0  88.9  0.439  11.3  34.9   
5    6.0    Los Angeles Clippers  19  240.0  39.3  85.3  0.461  14.1  36.1   
6    7.0           Atlanta Hawks  18  240.0  39.8  87.1  0.458  10.9  31.2   
7    8.0      Philadelphia 76ers  12  240.0  39.5  87.8  0.450   9.2  28.8   
8    9.0        Dallas Mavericks   7  240.0  39.0  80.0  0.488  13.4  36.0   
9   10.0               Utah Jazz  11  240.0  42.9  88.5  0.485  13.5  34.7   
10  11.0              Miami Heat   4  246.3  44.3  94.5  0.468  13.3  40.5   
11  12.0          Denver Nuggets  10  245.0  43.0  87.5  0.491  

In [27]:
seasons = range(2000, 2023)

get_advanced_team_stats(seasons)

In [28]:
get_advanced_team_stats(seasons, True)


HTTPError: HTTP Error 429: Too Many Requests